## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.head(10)

children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   
5  гражданский брак                 1      M   компаньон     0  255763.565419   
6   женат / замужем                 0      F   компаньон     0  240525.971920   
7   женат / замужем                 0      M   сотрудник     0  135823.934197   
8  гражданский брак                 1      F   сотрудник     0   95856.832424   
9   женат / замужем                 0      M   сотрудник     0  144425.938277   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  
3  дополнительное образование  
4             сыграть свадьбу  
5               покупка жилья  
6           операции с жильем  
7                 образование  
8       на проведение свадьбы  
9     покупка жилья для семьи

### Вывод

1. Файл с данными был прочтен и записан под именем **data**;

1. Вызоваем метод инфо к **data**
    1. Данные записаные в столбцы делятся на 3 типа. Неполная информация дана в столбцах *days_employed*, *total_income*
1. Информация по **data** была выведена вызовом  присвоенного имени c уточнением о выводе 30 верхних строк;
    1. Анализируюя данные можно сделать вывода ,что файл предстовляет собой таблицу состояющую из 12 столбцов, характеризующих информацию о клиенте банка, каждая строка отвечает набору данных об одном клиенте. всего 21525 строк, соответсвенно столько же и уникальных клинетов.
        1. Стобцец  *children* не вызывает вопросов.
        2. Столбец *days_employed* содержит как отрицательные значения так и гигантские положительные числа, которые наверняка являются ошибкой, ~~что с ней делать, кроме как удалить - непонятно.~~ **PS** просто заменим их на ноль
        3. Столбец *dob_years* не вызывает вопросов.
        4. Столбец *education* надо будет править, чтобы все названия были записаны прописными буквами
        5. Столбецы *family_status* и *family_status_id* несут одну информацию, вопросов не вызывают
        7. Столбцы  *gender*, *income_type*, *debt*   не вызывают вопросов.
        8. Столбецы *total_income* несет данные не в полном обьеме, надо править, либо удалять строки, либо менять на n -ые значения
        9. Столбец *purpose*, тут пригодится лемматизация, для корректной группировки данных.

### Шаг 2. Предобработка данных

In [4]:
data.groupby('income_type').mean()

children  days_employed  dob_years  education_id  \
income_type                                                         
безработный      0.500000  366413.652744  38.000000      0.500000   
в декрете        2.000000   -3296.759962  39.000000      1.000000   
госслужащий      0.622344   -3399.896902  40.636737      0.671693   
компаньон        0.598623   -2111.524398  39.697542      0.715438   
пенсионер        0.132002  365003.491245  59.063019      0.913900   
предприниматель  0.000000    -520.848083  42.500000      0.000000   
сотрудник        0.641784   -2326.499216  39.821027      0.849627   
студент          0.000000    -578.751554  22.000000      0.000000   

                 family_status_id      debt   total_income  
income_type                                                 
безработный              0.500000  0.500000  131339.751676  
в декрете                0.000000  1.000000   53829.130729  
госслужащий              0.857437  0.058944  170898.309923  
компаньон                1.032448  0.073943  202417.461462  
пенсионер                0.985737  0.056017  137127.465690  
предприниматель          0.500000  0.000000  499163.144947  
сотрудник                0.955661  0.095422  161380.260488  
студент                  4.000000  0.000000   98201.625314

In [5]:
data.groupby('income_type').count()

children  days_employed  dob_years  education  education_id  \
income_type                                                                    
безработный             2              2          2          2             2   
в декрете               1              1          1          1             1   
госслужащий          1459           1312       1459       1459          1459   
компаньон            5085           4577       5085       5085          5085   
пенсионер            3856           3443       3856       3856          3856   
предприниматель         2              1          2          2             2   
сотрудник           11119          10014      11119      11119         11119   
студент                 1              1          1          1             1   

                 family_status  family_status_id  gender   debt  total_income  \
income_type                                                                     
безработный                  2                 2       2      2             2   
в декрете                    1                 1       1      1             1   
госслужащий               1459              1459    1459   1459          1312   
компаньон                 5085              5085    5085   5085          4577   
пенсионер                 3856              3856    3856   3856          3443   
предприниматель              2                 2       2      2             1   
сотрудник                11119             11119   11119  11119         10014   
студент                      1                 1       1      1             1   

                 purpose  
income_type               
безработный            2  
в декрете              1  
госслужащий         1459  
компаньон           5085  
пенсионер           3856  
предприниматель        2  
сотрудник          11119  
студент                1

In [7]:
data.groupby('income_type').mean()

children  days_employed  dob_years  education_id  \
income_type                                                         
безработный      0.500000  366413.652744  38.000000      0.500000   
в декрете        2.000000   -3296.759962  39.000000      1.000000   
госслужащий      0.622344   -3399.896902  40.636737      0.671693   
компаньон        0.598623   -2111.524398  39.697542      0.715438   
пенсионер        0.132002  365003.491245  59.063019      0.913900   
предприниматель  0.000000    -520.848083  42.500000      0.000000   
сотрудник        0.641784   -2326.499216  39.821027      0.849627   
студент          0.000000    -578.751554  22.000000      0.000000   

                 family_status_id      debt   total_income  
income_type                                                 
безработный              0.500000  0.500000  131339.751676  
в декрете                0.000000  1.000000   53829.130729  
госслужащий              0.857437  0.058944  170898.309923  
компаньон                1.032448  0.073943  202417.461462  
пенсионер                0.985737  0.056017  137127.465690  
предприниматель          0.500000  0.000000  499163.144947  
сотрудник                0.955661  0.095422  161380.260488  
студент                  4.000000  0.000000   98201.625314

In [8]:
data.loc[data['days_employed'] > 0, 'days_employed'] = 0
data.head(10)

children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0       0.000000         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   
5  гражданский брак                 1      M   компаньон     0  255763.565419   
6   женат / замужем                 0      F   компаньон     0  240525.971920   
7   женат / замужем                 0      M   сотрудник     0  135823.934197   
8  гражданский брак                 1      F   сотрудник     0   95856.832424   
9   женат / замужем                 0      M   сотрудник     0  144425.938277   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  
3  дополнительное образование  
4             сыграть свадьбу  
5               покупка жилья  
6           операции с жильем  
7                 образование  
8       на проведение свадьбы  
9     покупка жилья для семьи

1. Cгруппировав и осреднив данные по столбцу *income_type* видно, что анломально большие значения принадлежат категории пенсионеры и безработный. Причина возникновения таких данных не ясна, возможно, данные по пенсионерам и безработным хранятся в отдельном месте и при выгрузке в общую таблицу происходит, то что происходит. 
1. Мы хотим привести все данные к общему виду, соответственно для начала посмотрим, сколько данных мы имеем в строках с пенсионерами и безработными, вызовем метод count к отгруппированной по *income_type* **data**. Общее колличество строк 21525, пенсионеров и безработных 3858, т.е если просто удалим эти данные то, потеряем 1/7 данных, что около 14%. Так как, конечной целью анализя является платежеспособность в зависимости от дохода, наличия детей или целей кредита, то отсутсвие хороших данных в столбцах *days_employed* не помешает нам сделать корректные выводы.
1. **PS** удалять строки c аномально большими значениями не будем, а вместо это заменим их на 0, через .loc и условию по столбцу *days_employed*


### Обработка пропусков

In [9]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [10]:
data.groupby('income_type')['days_employed'].apply(lambda x: x.isna().sum())

income_type
безработный           0
в декрете             0
госслужащий         147
компаньон           508
пенсионер           413
предприниматель       1
сотрудник          1105
студент               0
Name: days_employed, dtype: int64

Обработаем пропуска столбцов *days_employed*, *total_income*

In [11]:
data['days_employed'] = data.groupby('income_type')['days_employed'].apply(lambda x: x.fillna(x.mean()))
data.head(10)

children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0       0.000000         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   
5  гражданский брак                 1      M   компаньон     0  255763.565419   
6   женат / замужем                 0      F   компаньон     0  240525.971920   
7   женат / замужем                 0      M   сотрудник     0  135823.934197   
8  гражданский брак                 1      F   сотрудник     0   95856.832424   
9   женат / замужем                 0      M   сотрудник     0  144425.938277   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  
3  дополнительное образование  
4             сыграть свадьбу  
5               покупка жилья  
6           операции с жильем  
7                 образование  
8       на проведение свадьбы  
9     покупка жилья для семьи

In [12]:
data['total_income'] = data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.mean()))
data.head(10)

children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0       0.000000         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   
5  гражданский брак                 1      M   компаньон     0  255763.565419   
6   женат / замужем                 0      F   компаньон     0  240525.971920   
7   женат / замужем                 0      M   сотрудник     0  135823.934197   
8  гражданский брак                 1      F   сотрудник     0   95856.832424   
9   женат / замужем                 0      M   сотрудник     0  144425.938277   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  
3  дополнительное образование  
4             сыграть свадьбу  
5               покупка жилья  
6           операции с жильем  
7                 образование  
8       на проведение свадьбы  
9     покупка жилья для семьи

In [13]:
#data['days_employed'] = (data['days_employed']**2)**0.5
data['days_employed'] = data['days_employed'].apply(lambda x: abs(x))
data.head(10)

children  days_employed  dob_years education  education_id  \
0         1    8437.673028         42    высшее             0   
1         1    4024.803754         36   среднее             1   
2         0    5623.422610         33   Среднее             1   
3         3    4124.747207         32   среднее             1   
4         0       0.000000         53   среднее             1   
5         0     926.185831         27    высшее             0   
6         0    2879.202052         43    высшее             0   
7         0     152.779569         50   СРЕДНЕЕ             1   
8         2    6929.865299         35    ВЫСШЕЕ             0   
9         0    2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   
5  гражданский брак                 1      M   компаньон     0  255763.565419   
6   женат / замужем                 0      F   компаньон     0  240525.971920   
7   женат / замужем                 0      M   сотрудник     0  135823.934197   
8  гражданский брак                 1      F   сотрудник     0   95856.832424   
9   женат / замужем                 0      M   сотрудник     0  144425.938277   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  
3  дополнительное образование  
4             сыграть свадьбу  
5               покупка жилья  
6           операции с жильем  
7                 образование  
8       на проведение свадьбы  
9     покупка жилья для семьи

### Вывод

1. Посмотрим на колличество пропусков, вызываем методы *isna().sum()*. Имеем по ~~1761~~ 2174 пропускf в столбцах *days_employed* и *total_income*. (При первичном анализе было видно, что пропуска встречаются  для этих двух столбцов в одних и тех же строках)
1. Определим, где эти значения пропущены. Сгрупппируем по *'income_type'* вывыедем один столбец *days_employed* и применим к нему метод суммирования пропущенных значений. Получаем, что значения пропущенны равномерно у всех категорий гражд. Откуда взялись пропуски? МБ Ошибка при заполении форм в банке или ошибка выгрузки, однозначного вывода сделать нельзя.
1. Заменим пропуски средними значениями на группу. Группируем по типу доходов, и применяем метод fillna().mean(). Повторим операцию дважды, к столцам  *days_employed* и *total_income* соответсвенно. Конечно заполение пропусков средними - это самый простой метод, наверняка существуют более глубокие зависимости между данными, но мы пока не умеем их искать.
1. Данные по *days_employed* -отрицательны, сделаем их положительными, путем ~~возведенеия в квадрат и извлечения корня.~~ применения функции abs.

### Замена типа данных

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [16]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [18]:
data.sum()

children                                                        11600
days_employed                                                41561925
dob_years                                                      931890
education           высшеесреднееСреднеесреднеесреднеевысшеевысшее...
education_id                                                    17591
family_status       женат / замужемженат / замужемженат / замужемж...
family_status_id                                                20934
gender              FFMMFMFMFMMFMFFFFMFFMFFMFMMMFFFFFFFMMFMFMFFFFF...
income_type         сотрудниксотрудниксотрудниксотрудникпенсионерк...
debt                                                             1741
total_income                                               3603186641
purpose             покупка жильяприобретение автомобиляпокупка жи...
dtype: object

In [19]:
data.head()

children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   Среднее             1   
3         3           4124         32   среднее             1   
4         0              0         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  
3  дополнительное образование  
4             сыграть свадьбу

### Вывод

1. Еще раз посмотрели типы данных по столбцам, как и раньше два столбца имеют вещественный тип данных это: *days_employed* и *total_income*
1. Для того чтобы убедиться, что все числа - числа , просуммируем столбы методом sum(). Все числа - числа.
1. Преобразуем вещественный тип данных в целочисленный методом astype('int') для вышеуказанных столбцов. Такой метод даст нам на выходе int64 - формат целочисленных данных в давайфрейми.
1. Еще раз выведем тип данных и  просуммируем по столбцам. Все получилось.

### Обработка дубликатов

In [20]:
data.head()

children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   Среднее             1   
3         3           4124         32   среднее             1   
4         0              0         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  
3  дополнительное образование  
4             сыграть свадьбу

In [21]:
data['education'] = data['education'].astype(str).str.lower()

In [22]:
data.head(10)

children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0              0         53   среднее             1   
5         0            926         27    высшее             0   
6         0           2879         43    высшее             0   
7         0            152         50   среднее             1   
8         2           6929         35    высшее             0   
9         0           2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   
5  гражданский брак                 1      M   компаньон     0        255763   
6   женат / замужем                 0      F   компаньон     0        240525   
7   женат / замужем                 0      M   сотрудник     0        135823   
8  гражданский брак                 1      F   сотрудник     0         95856   
9   женат / замужем                 0      M   сотрудник     0        144425   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  
3  дополнительное образование  
4             сыграть свадьбу  
5               покупка жилья  
6           операции с жильем  
7                 образование  
8       на проведение свадьбы  
9     покупка жилья для семьи

In [23]:
data.duplicated().sum()

71

In [24]:
data = data.drop_duplicates().reset_index(drop = True)

In [25]:
data.duplicated().sum()

0

### Вывод

1. Перед обработкой дубликатов, оценим, есть ли столбцы, где данные записаны разным регистром. Вызываем первые строки, 
видим что в столбце *education* есть такие данные.
1. Применим к столбцу *education* метод aslower(str) предорительно поменяв тип данных на str.
1. Считаем колличество пропуском методом **.duplicated().sum()** . У нас их 71. Возможные причины появления дубликатов в данных - данные об одном клиенте были внесены несколько раз или прокопированы в процессе сбора. Особый случай представляют дупликаты которые имеют разные регистр слов в столбце *education*, такие данные возможно могли появиться при ручном заполнении информации о клиенте;
1. Дропаем дупликаты методом **drop_duplicates** и ресетим индексы.
1. Смотрим сумму дупликатов, она равна 0

### Лемматизация

In [26]:
from pymystem3 import Mystem
m = Mystem()

In [27]:
data['purpose'] = data['purpose'].apply(lambda x: ','.join(m.lemmatize(x)))

In [28]:
data.head()

children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0              0         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

                           purpose  
0               покупка, ,жилье,\n  
1     приобретение, ,автомобиль,\n  
2               покупка, ,жилье,\n  
3  дополнительный, ,образование,\n  
4             сыграть, ,свадьба,\n

In [29]:
data['purpose'].value_counts()

автомобиль,\n                                    972
свадьба,\n                                       791
на, ,проведение, ,свадьба,\n                     768
сыграть, ,свадьба,\n                             765
операция, ,с, ,недвижимость,\n                   675
покупка, ,коммерческий, ,недвижимость,\n         661
операция, ,с, ,жилье,\n                          652
покупка, ,жилье, ,для, ,сдача,\n                 651
операция, ,с, ,коммерческий, ,недвижимость,\n    650
жилье,\n                                         646
покупка, ,жилье,\n                               646
покупка, ,жилье, ,для, ,семья,\n                 638
строительство, ,собственный, ,недвижимость,\n    635
недвижимость,\n                                  633
операция, ,со, ,свой, ,недвижимость,\n           627
строительство, ,жилой, ,недвижимость,\n          624
покупка, ,недвижимость,\n                        621
покупка, ,свой, ,жилье,\n                        620
строительство, ,недвижимость,\n               

In [31]:
purpose_list = ['жилье', 'недвижимость', 'автомобиль', 'образование' ,'свадьба']
def purpose_mod(purpose):
    for i in purpose_list:
        if i in purpose:
            return i

In [32]:
data['purpose']= data['purpose'].apply(purpose_mod)
data.head(10)

children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0              0         53   среднее             1   
5         0            926         27    высшее             0   
6         0           2879         43    высшее             0   
7         0            152         50   среднее             1   
8         2           6929         35    высшее             0   
9         0           2188         41   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   
5  гражданский брак                 1      M   компаньон     0        255763   
6   женат / замужем                 0      F   компаньон     0        240525   
7   женат / замужем                 0      M   сотрудник     0        135823   
8  гражданский брак                 1      F   сотрудник     0         95856   
9   женат / замужем                 0      M   сотрудник     0        144425   

       purpose  
0        жилье  
1   автомобиль  
2        жилье  
3  образование  
4      свадьба  
5        жилье  
6        жилье  
7  образование  
8      свадьба  
9        жилье

In [33]:
data['purpose'].value_counts()

недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose, dtype: int64

### Вывод

1. Перед началом лемантизации подгружаем библиотеку  **pymystem3**, импортируем **Mystem**
2. Проводим операцию лемантизации к столбу *purpose* , с методом *join*
3. Смотрим что получилось, методом *value_counts*. Видно, что не смотря на высокую захламленность данных, выделяется 5 основных целей на которые люди хотят взять кредит это; жилье, недвижимость, автомобиль, образование и свадьба
4. ~~Легким движением руки, очищаем столбец *purpose* от ненужной информации методом *replace*~~ Чтобы избавить столбец purpose от ненужных символов, создадим функцию purpose_mod в которую будет подаваться каждый элемент списка purpose_list и проверяться по условию для каждой строки. Если условие проходит то значение элемента i присваивается строке.
5. Проверяем результат методом *value_counts*, видим, что остались нужные нам 5 подкатигорий, по которым мы дальше будем категоризировать и делать выводы.

**P.S** в этом блоке происходит и леммантизация и чуть читска данных для категоризации, не стал двигать код в категоризацию данных, чтобы было видно как сделал исправления.

### Категоризация данных

In [34]:
data['total_income'].mean()

167431.58371399273

In [35]:
def income_status (total_income):
    if total_income < 167431:
        return 'ниже среднего'
    else:
        return 'выше среднего'

In [36]:
data['income_status']= data['total_income'].apply(income_status)

In [37]:
data.head()

children  days_employed  dob_years education  education_id  \
0         1           8437         42    высшее             0   
1         1           4024         36   среднее             1   
2         0           5623         33   среднее             1   
3         3           4124         32   среднее             1   
4         0              0         53   среднее             1   

      family_status  family_status_id gender income_type  debt  total_income  \
0   женат / замужем                 0      F   сотрудник     0        253875   
1   женат / замужем                 0      F   сотрудник     0        112080   
2   женат / замужем                 0      M   сотрудник     0        145885   
3   женат / замужем                 0      M   сотрудник     0        267628   
4  гражданский брак                 1      F   пенсионер     0        158616   

       purpose  income_status  
0        жилье  выше среднего  
1   автомобиль  ниже среднего  
2        жилье  ниже среднего  
3  образование  выше среднего  
4      свадьба  ниже среднего

### Вывод

1. Категоризуем данные по признаку достатка. Посчитаем среднее, методом *mean*, от этого числа будем строить нашу дальнейшую категоризацию
2. Разделим категории граждан на имеющих низкий и выский доход соответсвенно, сделаем это путем ввода функции *income_status* по условию среднего дохода. Если меньше среднего то доход ниже среднего, выше - выше среднего. 
3. С помощью метода *apply* применим функцию *income_status* , результаты который поместим в новый столбец с именем *income_status* 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [38]:
data_children_debt_zero = data[(data['children']== 1) & (data['debt'] == 0)]['children'].count()
data_children_debt_one = data[(data['children']== 1) & (data['debt'] == 1)]['children'].count()
check_children = data_children_debt_one/(data_children_debt_zero + data_children_debt_one)
#_____________________


data_nochildren_debt_zero = data[(data['children']== 0) & (data['debt'] == 0)]['children'].count()
data_nochildren_debt_one = data[(data['children']== 0) & (data['debt'] == 1)]['children'].count()
check_nochildren = data_nochildren_debt_one/(data_nochildren_debt_zero + data_nochildren_debt_one )
#_____________________

print('Процент невозврата кредита если есть ребенок: {:.1%}' .format(check_children))
print('Процент невозврата кредита если нет детей: {:.1%}' .format(check_nochildren))

Процент невозврата кредита если есть ребенок: 9.2%
Процент невозврата кредита если нет детей: 7.5%


### Вывод

**Кажется, кредит возвращают лучше, если детей нет.**

Такой вывод мы сделали после следующих операций:
1. Найдем колличество кредитующихся по условию наличия/отсутствия  детей и кредитов соответственно. Для этого сделаем 4 датафрейма, путем выстовления соответсвующей комбинации условий. Например для data_children_debt_zero условия будут **(data['children']== 1) & (data['debt'] == 0)**. Далее посчитаем число получившихся строк методом *count()*
1. Аналогично проделаем ту же операцию для других возможных сочетаний наличия детей и долгов.
1. Посчитаем отношения для людей которые не вернули кредит к сумме всех людей, для каждого случая (есть дети, нет детей)
1. Выведем полученные данные функцией *print*


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
data_pivot = data.pivot_table(index=['family_status'], values='debt', aggfunc='sum')
print(data_pivot.head(10))

                       debt
family_status              
Не женат / не замужем   274
в разводе                85
вдовец / вдова           63
гражданский брак        388
женат / замужем         931


In [40]:
data_pivot_2 = data.groupby('family_status')['debt'].count()
print(data_pivot_2.head(10))

family_status
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
гражданский брак          4151
женат / замужем          12339
Name: debt, dtype: int64


In [41]:
data_pivot['debt_sum'] = data_pivot_2
data_pivot['%_of_noreturn'] = data_pivot['debt'] / data_pivot['debt_sum']*100
data_pivot.sort_values(by = '%_of_noreturn', ascending = False)

debt  debt_sum  %_of_noreturn
family_status                                       
Не женат / не замужем   274      2810       9.750890
гражданский брак        388      4151       9.347145
женат / замужем         931     12339       7.545182
в разводе                85      1195       7.112971
вдовец / вдова           63       959       6.569343

### Вывод

**Кажется, семейное положение не влияет на возврат кредита.**

Такой вывод мы сделали после следующих операций:
1. Создадим  *data_pivot* методом pivot_table по столбу family_status и сосчитаем колличество людей с долгами через *'sum'*. Получили колличество должников по каждой категории
1. Создадим *data_pivot_2* методом groupby и посчитаем общее колличество людей в каждой категории
1. Обьединим *data_pivot* и *data_pivot_2* путем присвоения новому столбцу['debt_sum'] в датафреме  *data_pivot* значение датафрейма *data_pivot_2*
1. Уже в обьедененной таблице создадим столбец *%_of_noreturn* путем деления *debt* на *debt_sum*. Отсортируем по убываю.
1. *Выводы не однозначны, не женатые не возращают кредит на ровне с теми, кто живет в гражданском браке. А женатые и в разводе делают это на 2% меньше. Вдовцы/вдовы делают долги меньше всех.*

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [42]:
income_below_av_debt_zero = data[(data['income_status']== 'ниже среднего') & (data['debt'] == 0)]['income_status'].count()
income_below_av_debt_one = data[(data['income_status']== 'ниже среднего') & (data['debt'] == 1)]['income_status'].count()
check_income_b = income_below_av_debt_one/(income_below_av_debt_zero + income_below_av_debt_one)

income_above_av_debt_zero = data[(data['income_status']== 'выше среднего') & (data['debt'] == 0)]['income_status'].count()
income_above_av_debt_one = data[(data['income_status']== 'выше среднего') & (data['debt'] == 1)]['income_status'].count()
check_income_a = income_above_av_debt_one/(income_above_av_debt_zero + income_above_av_debt_one)

print('Процент невозврата кредита у клиентов с низким зароботком: {:.1%}' .format(check_income_b))
print('Процент невозврата кредита у клиентов с высоким зароботком: {:.1%}' .format(check_income_a))


Процент невозврата кредита у клиентов с низким зароботком: 8.5%
Процент невозврата кредита у клиентов с высоким зароботком: 7.5%


### Вывод

**Кажется, кредит возвращают лучше когда доход больше.**

1. Такой вывод мы сделали после операций, которые аналогичны для 1 случая (отсутствие или наличие детей).
1. Вывод логичен, вопросов не вызывает.


- Как разные цели кредита влияют на его возврат в срок?

In [43]:
data_pivot_purpose = data.pivot_table(index=['purpose'], columns='debt', values='total_income', aggfunc='count')
data_pivot_purpose['%_of_noreturn'] = data_pivot_purpose[1] / (data_pivot_purpose[0]+data_pivot_purpose[1]) *100
data_pivot_purpose.sort_values(by = '%_of_noreturn', ascending = False)

debt             0    1  %_of_noreturn
purpose                               
автомобиль    3903  403       9.359034
образование   3643  370       9.220035
свадьба       2138  186       8.003442
недвижимость  5877  474       7.463392
жилье         4152  308       6.905830

### Вывод

**Кажется, кредит возвращают лучше когда покупают жилье, хуже - при покупки автомобиля.**

Такой вывод мы сделали после следующих операций:
1. Создадим  *data_pivot_purpose* методом pivot_table по столбу purpose и сосчитаем колличество людей с долгами через *'sum'*. Получили как колличество должников, так и тех кто возвращает кредит.
1. Создадим столбец *%_of_noreturn* путем деления должников на сумму должников и тех кто возвращает кридит. Отсортируем таблицу по убываю.


### Шаг 4. Общий вывод

1. Мы проделали большую работу. Условно результат нашего анализа можно поместить в несколько пунктов.
    1. Не возвращают кредит все, вне зависимости от наличия детей, дохода и целей кредита, кто-то хуже, кто-то лучше.
    1. В паре по признаку наличия детей, проигрывают те, у кого они есть. Наверняка сказывается дополонительная финансовая нагрузка в связи с появлением ребенка.
    1. Наличие партнера не сказывается на то, вернет человек кредит или нет. Тут мы не нашли прямой корреляции.
    1. Как и ожидалось, если есть деньги, то кредит возвращается проще.
    1. Автомобили берут в кредит много, но так же и не возвращают кредит больше всего по этому пункту. То же самое с образованием, видимо наличие диплома не помогает устроиться на более высокооплачиваемую работу. Зато с жильем все предельно осторожны, конечно, ведь пока ты не отдашь кредит, банк фактически владеет твоей квартирой.
    
**Конец**

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.